In [1]:
from model.vit import VisionTransformer

In [4]:
plt.cm.tab20(3 /13)[:3]

(0.17254901960784313, 0.6274509803921569, 0.17254901960784313)

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Simuler des données pour l'exemple
np.random.seed(42)  # Pour la reproductibilité

# Dimensions de l'image et des masques
height, width = 256, 256
num_classes = 13

# Créer une image RGB synthétique
image = np.random.rand(height, width, 3)

# Créer une prédiction et une vérité terrain synthétiques
prediction = np.random.randint(0, num_classes, size=(height, width))
target_view = np.random.randint(0, num_classes, size=(height, width))

# Labels de classe (doit correspondre à votre exemple)
class_labels = {
    0: 'building',
    1: 'pervious surface',
    2: 'impervious surface',
    3: 'bare soil',
    4: 'water',
    5: 'coniferous',
    6: 'deciduous',
    7: 'brushwood',
    8: 'vineyard',
    9: 'herbaceous vegetation',
    10: 'agricultural land',
    11: 'plowed land',
    12: 'other'
}

# Génération de la légende
legend_patches = [mpatches.Patch(color=plt.cm.tab20(i / num_classes), label=class_labels[i]) for i in class_labels]

# Fonction pour superposer une segmentation sur une image
def overlay_segmentation(image, segmentation, alpha=0.5):
    overlay = image.copy()
    for class_id, class_name in class_labels.items():
        color = plt.cm.tab20(class_id / num_classes)[:3]  # Obtenir les 3 premières valeurs (RGB)
        overlay[segmentation == class_id] = color
    return (1 - alpha) * image + alpha * overlay

# Superposer les prédictions et la vérité terrain
overlay_pred = overlay_segmentation(image, prediction)
overlay_target = overlay_segmentation(image, target_view)

# Tracer les images
fig, axs = plt.subplots(1, 2, figsize=(15, 7.5))

# Image avec prédictions superposées
axs[0].imshow(overlay_pred)
axs[0].set_title('Prédictions Superposées')
axs[0].axis('off')

# Image avec vérité terrain superposée
axs[1].imshow(overlay_target)
axs[1].set_title('Vérité Terrain Superposée')
axs[1].axis('off')

# Ajouter la légende
fig.legend(handles=legend_patches, loc='upper center', ncol=4, bbox_to_anchor=(0.5, 0.11), fontsize='small')

# Sauvegarder l'image dans un fichier .png
plt.savefig('segmentation_overlay_demo.png', bbox_inches='tight', dpi=300)
plt.show()

In [2]:
vit = VisionTransformer(
                image_size = 1024,
                patch_size = 16,
                n_layers = 32,
                d_model = 1280,
                d_ff = 4*1280,
                n_heads = 16,
                n_cls = 13,
                dropout=0.1,
                drop_path_rate=0.0,
                distilled=False,
                channels=3,
            )

In [4]:
import torch
chckpts = torch.load("/run/user/108646/gvfs/sftp:host=flexo/d/maboum/JSTARS/segmentation/checkpoints/sam_vit_h_4b8939.pth")
#vit.load_state_dict(chckpts)

In [5]:
# Modify the keys
new_checkpoint = {}
for key, value in chckpts.items():    
    new_key = key.replace("image_encoder.", "")  # Replace "image_encoder" with an empty string
    new_key = new_key.replace(".lin", ".fc")
    #print(new_key)
    new_checkpoint[new_key] = value
new_checkpoint.keys()
# Save the modified checkpoint
#torch.save(new_checkpoint, "/d/maboum/JSTARS/segmentation/checkpoints/sam_vit_h_4b8939v2.pth")


dict_keys(['neck.0.weight', 'neck.1.weight', 'neck.1.bias', 'neck.2.weight', 'neck.3.weight', 'neck.3.bias', 'patch_embed.proj.weight', 'patch_embed.proj.bias', 'blocks.0.norm1.weight', 'blocks.0.norm1.bias', 'blocks.0.attn.rel_pos_h', 'blocks.0.attn.rel_pos_w', 'blocks.0.attn.qkv.weight', 'blocks.0.attn.qkv.bias', 'blocks.0.attn.proj.weight', 'blocks.0.attn.proj.bias', 'blocks.0.norm2.weight', 'blocks.0.norm2.bias', 'blocks.0.mlp.fc1.weight', 'blocks.0.mlp.fc1.bias', 'blocks.0.mlp.fc2.weight', 'blocks.0.mlp.fc2.bias', 'blocks.1.norm1.weight', 'blocks.1.norm1.bias', 'blocks.1.attn.rel_pos_h', 'blocks.1.attn.rel_pos_w', 'blocks.1.attn.qkv.weight', 'blocks.1.attn.qkv.bias', 'blocks.1.attn.proj.weight', 'blocks.1.attn.proj.bias', 'blocks.1.norm2.weight', 'blocks.1.norm2.bias', 'blocks.1.mlp.fc1.weight', 'blocks.1.mlp.fc1.bias', 'blocks.1.mlp.fc2.weight', 'blocks.1.mlp.fc2.bias', 'blocks.2.norm1.weight', 'blocks.2.norm1.bias', 'blocks.2.attn.rel_pos_h', 'blocks.2.attn.rel_pos_w', 'blocks.2

In [8]:
vit.load_state_dict(new_checkpoint,False)

RuntimeError: Error(s) in loading state_dict for VisionTransformer:
	size mismatch for pos_embed: copying a param with shape torch.Size([1, 64, 64, 1280]) from checkpoint, the shape in current model is torch.Size([1, 4097, 1280]).

In [10]:
new_checkpoint['pos_embed'].shape

torch.Size([1, 64, 64, 1280])

In [ ]:
mask_decoder_keys_to_remove = ['key1', 'key2', 'key3']  # Replace with the keys you want to remove from mask_decoder
prompt_encoder_keys_to_remove = ['key4', 'key5', 'key6']  # Replace with the keys you want to remove from prompt_encoder

for key in mask_decoder_keys_to_remove:
    mask_decoder.pop(key, None)

for key in prompt_encoder_keys_to_remove:
    prompt_encoder.pop(key, None)
